<a href="https://colab.research.google.com/github/sayandas96476/RAG/blob/main/RAG_Dense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a RAG application
================================
Data source: Wikipedia API(Parsed using Beautiful Soup)

Retriever: Dense Retrieval used(E5 Large model)

Generator: Prompt used(mixtral-8x7b-32768)



WORKFLOW
================================================================

Here I have created externel knowledge base source using Wikipedia

I have used BeautifulSoup to parse the content from wikipedia based title which I reived using requests library

Then the document is preprocessed. I have used e5-large model as the embedding model as we were getting large chunk size

We have created Chroma as our vector database.Then stored it in google drive location

In the next step we load the vector database and extract top 3 most similar records

We pass it to the generator which gives us the result


CONCLUSION:
==================================================================
When compared to Sparse Retrieval, Dense Retrieval fares better.

Some queries which were not answered clearly by Sparse is articulately answered by Dense.

In [2]:

from google.colab import drive
drive.mount('/content/drive')


!pip install -q langchain-groq langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 2.7 MB/s eta 0:00:00


In [3]:
!pip install chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00


In [4]:
!pip install -q langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.5 MB/s eta 0:00:00


In [7]:

import chromadb
import requests
from bs4 import BeautifulSoup

from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
import os
from langchain_community.vectorstores import Chroma

from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader

from langchain.text_splitter import CharacterTextSplitter

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
os.environ["GROQ_API_KEY"] = ""  # Replace with your actual API key

llm = ChatGroq(
    model_name="mixtral-8x7b-32768",  # You can also use "llama2-70b-4096"
    temperature=0.7,
    max_tokens=1024
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
#obtained data using wikipedia api
#prsed using beautiful soup

def get_full_wikipedia_content(title):
    # Construct the URL for the full Wikipedia page
    url = f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"

    try:
        # Make the request
        response = requests.get(url)
        response.raise_for_status()

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the main content div
        content_div = soup.find(id="mw-content-text")

        # Extract all paragraphs
        paragraphs = content_div.find_all('p')

        # Combine all paragraph texts
        full_text = '\n\n'.join([para.get_text() for para in paragraphs])

        # Remove citations [1], [2], etc.
        import re
        full_text = re.sub(r'\[\d+\]', '', full_text)

        return full_text.strip()

    except requests.RequestException as e:
        return f"Error fetching page: {str(e)}"


#preprocessing
def combine_strings(original_list, chunk_size=3):
    return [''.join(original_list[i:i + chunk_size])
            for i in range(0, len(original_list), chunk_size)]


def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize
    words = text.split()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if w not in stop_words]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)



In [9]:
#obtaining text
title = "Batman"
text = get_full_wikipedia_content(title)
text = text.replace('\n', '')

lis = text.split(".")

combined = combine_strings(lis)
text = """ """
for i in combined:
  text += i+"\n\n\n"


In [10]:

#chunking
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = '\n\n\n',
    chunk_size = 200,
)


In [11]:
#embedding to be done
embedding_function = SentenceTransformerEmbeddings(model_name= "intfloat/e5-large")



<ipython-input-11-7537ac681c83>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name= "intfloat/e5-large")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datas

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [12]:
#creation of vector database
texts = text_splitter.create_documents([text])
persist_directory = "/content/drive/MyDrive/testing/db_test"
vectordb = Chroma.from_documents(documents=texts, embedding=embedding_function, persist_directory=persist_directory)

In [13]:

#loading the vector database
vectordb = Chroma(
    persist_directory="/content/drive/MyDrive/testing/db_test",
    embedding_function=embedding_function
)

<ipython-input-13-eac6614780a5>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [21]:
#getting context
def retrieve(query):
  docs = vectordb.similarity_search(query,k=3)
  for i in docs:
    CONTEXT = i.page_content + "\n==============\n"
  return CONTEXT

#generation
def generate(CONTEXT,query):
  template=prompt = f"""
  use the following CONTEXT to answer the QUESTION at the end.
  If you don't know the answer, just say that I don't know, don't try to make up an answer.
  Also remember dont use any external knowledge and only refer to this CONTEXT to answer question.
  Consider this CONTEXT as the ultimate truth.

  CONTEXT: {CONTEXT}
  QUESTION: {query}

  """
  prompt = PromptTemplate(
      input_variables=["query","CONTEXT"],
      template=template
  )

  chain = LLMChain(llm=llm, prompt=prompt)
  response = chain.invoke({"query": query, "CONTEXT":CONTEXT})
  return response['text']






In [22]:
query = "Who created Batman comic"
CONTEXT = retrieve(query)
answer = generate(CONTEXT,query)
print(answer)

Based on the provided context, Bob Kane conceived the idea for Batman, but Bill Finger substantially developed the concept. Therefore, it's more accurate to say that Batman was a creation of both Kane and Finger, although Kane often claimed sole credit.


In [23]:
query = "Who was batmans girlfriend"
CONTEXT = retrieve(query)
answer = generate(CONTEXT,query)
print(answer)

Based on the information provided in the context, Batman's girlfriend was Julie Madison. It's worth noting that their relationship likely occurred during the early stages of Batman's career, before he became more widely accepted by the police force.


In [24]:
query = "Any idea on Batman's love life"
CONTEXT = retrieve(query)
answer = generate(CONTEXT,query)
print(answer)

Yes, based on the provided context, Batman has a complicated romantic involvement with Vicki and a relationship with Talia al Ghul. However, his relationship with Talia is fraught with conflict due to her father's criminal ambitions.


In [25]:
query = "Who was Batman's most ferocious enemy?"
CONTEXT = retrieve(query)
answer = generate(CONTEXT,query)
print(answer)

Based on the information provided in the context, Batman's most formidable and often referred to as his "archenemy" is the Joker. Therefore, the answer to the question is the Joker.
